In [1]:
from pseudo_IFD_tsne import *
#from utils import load_data, MidpointNormalize, equipotential_standard_normal, plot_heatmaps
from utils import MidpointNormalize, load_data
import matplotlib.pyplot as plt
import tikzplotlib

from tueplots import cycler, fonts, fontsizes, bundles
from tueplots.constants import markers
from tueplots.constants.color import palettes

plt.rcParams.update(cycler.cycler(color=palettes.tue_plot))
plt.rcParams.update(fonts.aistats2022_tex(family="serif"))
plt.rcParams.update(fontsizes.aistats2022())
plt.rcParams['axes.grid'] = False

In [2]:
def KL_divergence(X_flat, Y_flat, X_unflattener, Y_unflattener):
    """
    (R^nxp x R^nxp)--> R
    """
    X = X_unflattener(X_flat)
    Y = Y_unflattener(Y_flat)
    learning_rate, perplexity = (200, 30.0)
    P = x2p(X, tol=1e-5, perplexity=perplexity)
    P = (P + np.transpose(P))
    P = P / np.sum(P)      # Why don't we devide by 2N as described everywhere?
    P = np.maximum(P, 1e-12)
    Q, _ = y2q(Y)
    return np.sum(P * (np.log(P+1e-10) - np.log(Q+1e-10)))

In [3]:
#X, y = make_blobs(n_samples=50, n_features=50, centers=4, random_state=0, shuffle=False, cluster_std=[0.1, 3, 3, 3])
X, y = load_data(50)
key = random.PRNGKey(41)
#X = onp.array(random.normal(key, shape=(50, 50)))
y_guess = random.normal(key, shape=(X.shape[0], 2))
#Y_star = TSNE(n_components=2, learning_rate=200, init=onp.array(y_guess), perplexity=30).fit_transform(X)
Y_star = tsne_fwd(X, y_guess)

X_flat, X_unflattener = flatten_util.ravel_pytree(X)   # row-wise
Y_flat, Y_unflattener = flatten_util.ravel_pytree(Y_star)

primals = (X_flat, Y_flat)
KL_divergence_fun = partial(KL_divergence, X_unflattener=X_unflattener, Y_unflattener= Y_unflattener)


I0000 00:00:1713189990.405930       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


===> Finding 49 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 0.01 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.00 seconds


In [4]:
fun = lambda y: KL_divergence(X_flat, y, X_unflattener, Y_unflattener)
H = jax.hessian(fun)(Y_flat)

Computing pairwise distances...
Starting binary search
Entered binary search function


In [5]:
H_pinv = np.linalg.pinv(H, hermitian=True)

In [7]:
dy, H, J = d_y_star_d_x_outer(KL_divergence, X, Y_star)
print(dy)

Computing pairwise distances...
Starting binary search
Entered binary search function
Computing pairwise distances...
Starting binary search
Entered binary search function
[[ 2.0722039e-02  1.5057951e-01 -9.8503819e-03 ... -1.4805503e-03
   1.7707280e-03  6.0440018e-04]
 [ 6.3704476e-02 -1.6220343e-01  2.2229955e-01 ... -2.0348813e-04
   4.5886193e-03 -1.1989415e-03]
 [ 3.9617132e-02  1.5541990e-02 -9.4627598e-03 ... -6.2873559e-03
   1.4627682e-03 -3.0590252e-03]
 ...
 [ 1.4788301e-02 -2.2593169e-02  1.1844771e-02 ...  6.7093177e-04
   2.8862339e-03  1.5237431e-04]
 [-7.7889685e-04 -7.1484102e-03  1.1437444e-02 ... -3.1381760e-02
  -1.2881519e-02  6.1443285e-03]
 [ 2.8236836e-02  2.4716349e-02 -1.8759809e-02 ... -2.0767743e-02
   2.2039216e-02 -3.9588530e-02]]


In [15]:
M = np.eye(len(Y_flat))
time_start = time.time()
dy_dx = M_d_y_star_d_x_P(KL_divergence_fun, primals, M, H_pinv)
time_end = time.time()
print(time_end - time_start)

Compute v3
Computing pairwise distances...
Starting binary search
Entered binary search function
21.63188099861145


In [16]:
print(dy_dx)

[[ 2.1332193e-02  1.5068763e-01 -9.7194500e-03 ... -1.5453796e-03
   1.8262810e-03  5.4034288e-04]
 [ 6.5048568e-02 -1.6308565e-01  2.2328450e-01 ... -2.9386231e-04
   4.7278106e-03 -1.3226976e-03]
 [ 4.0290311e-02  1.5672728e-02 -9.4432291e-03 ... -6.3919667e-03
   1.5151039e-03 -3.1401573e-03]
 ...
 [ 1.5284296e-02 -2.3027856e-02  1.2217485e-02 ...  6.4227590e-04
   2.9427425e-03  1.0945520e-04]
 [-1.1989936e-03 -7.0138369e-03  1.1357425e-02 ... -3.1405289e-02
  -1.2952458e-02  6.1942330e-03]
 [ 2.8961241e-02  2.4745701e-02 -1.8697346e-02 ... -2.0916153e-02
   2.2118343e-02 -3.9738230e-02]]


In [14]:
v = np.eye(len(Y_flat))[0]
time_start = time.time()
dy_dx = V_d_y_star_d_x_P(KL_divergence_fun, primals, v, H_pinv)
time_end = time.time()
print(time_end - time_start)

Compute v3
Computing pairwise distances...
Starting binary search
Entered binary search function
20.373340845108032


In [12]:
print(dy_dx)

[ 0.02133219  0.15068763 -0.00971945 ... -0.00154538  0.00182628
  0.00054034]
